# Scope 3 GHG Emission Data Preperation
#### - *Cleaning and Normalizing Data*
#### - *Merging World Bank Data to Scope 3 GHG Emission Data*

***

Import libraries and load csv data into a Dataframe

In [16]:
import wbgapi as wb
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') #turn off warnings

%matplotlib inline

df = pd.read_csv('All_Data_Start.csv')
df.head()

,account_id,account_name,incorporated_country,ticker,Evaluation_status,Scope_3_emissions_type,Scope_3_emissions_amount,Symbol_1,Year,Market_Cap,...,totalInvestments_USD,totalDebt_USD,totalEquity_USD,environmentalScore,socialScore,governanceScore,ESGScore,employeeCount,random_uniform,random_normal
0,44,3i Group,United Kingdom,III LN,"Relevant, calculated",Waste generated in operations,12.34,III.L,2013,3.654278e+09,...,4.460240e+09,1.078230e+09,4.201160e+09,NaN,NaN,NaN,NaN,NaN,0.568078,-2.373542
1,44,3i Group,United Kingdom,III LN,"Relevant, calculated",Business travel,56.78,III.L,2013,3.654278e+09,...,4.460240e+09,1.078230e+09,4.201160e+09,NaN,NaN,NaN,NaN,NaN,0.149474,0.650400
2,53,Abertis Infraestructuras,Spain,ABE SM,"Relevant, calculated",Purchased goods and services,53635.12,ABE.MC,2013,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.337876,-0.689242
3,53,Abertis Infraestructuras,Spain,ABE SM,"Relevant, calculated",Waste generated in operations,2518.18,ABE.MC,2013,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.323919,0.207426
4,53,Abertis Infraestructuras,Spain,ABE SM,"Relevant, calculated",Business travel,913.42,ABE.MC,2013,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.988926,0.337583


In [18]:
# verify number of columns and rows
df.shape

(181326, 49)

In [36]:
# drop columns
columns_to_drop = ['Symbol_1',
'random_normal', 
'random_uniform', 
'ESGScore',
'environmentalScore',
'socialScore',
'governanceScore',]
df = df.drop(columns_to_drop, axis=1)

### *Summary Dataframe*

In [42]:
# Create a summary DataFrame with column statistics
pd.set_option('display.float_format', lambda x: '%.0f' % x)  

summary_df = pd.DataFrame({
    'column_name': df.columns,
    'data_type': df.dtypes,
    'min_value': [df[col].min() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
    'max_value': [df[col].max() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
    'mean': [df[col].mean() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
    'median': [df[col].median() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
    'std_dev': [df[col].std() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
    'unique_count': df.nunique(),
    'null_count': df.isnull().sum()
})

# Reset index to make it cleaner
summary_df = summary_df.reset_index(drop=True)
summary_df

,column_name,data_type,min_value,max_value,mean,median,std_dev,unique_count,null_count
0,account_id,int64,26,1003410,141359,19845,292755,8795,0
1,account_name,object,NaN,NaN,NaN,NaN,NaN,9343,0
2,incorporated_country,object,NaN,NaN,NaN,NaN,NaN,113,0
3,ticker,object,NaN,NaN,NaN,NaN,NaN,6565,63209
4,Evaluation_status,object,NaN,NaN,NaN,NaN,NaN,5,849
5,Scope_3_emissions_type,object,NaN,NaN,NaN,NaN,NaN,17,0
6,Scope_3_emissions_amount,float64,0,121028620297,3438165,3597,303721401,89989,0
7,Year,int64,2013,2023,2019,2020,3,11,0
8,Market_Cap,float64,65,11293245766434816000,1655214327106429,33370260000,136663140912437136,17223,65255
9,Employee_Count,float64,0,12200000,47616,12000,193230,1605,157759


### Normalize Country Names

In [44]:
# get all unique country values
df['incorporated_country'].unique()

array(['United Kingdom', 'Spain', 'USA', 'Italy', 'South Africa', 'India',
       'France', 'Switzerland', 'Netherlands', 'Japan', 'Brazil',
       'Finland', 'Canada', 'New Zealand', 'Turkey', 'Germany', 'Ireland',
       'Australia', 'Austria', 'Luxembourg', 'Taiwan', 'Norway', 'Sweden',
       'Portugal', 'Argentina', 'Denmark', 'Belgium', 'Hong Kong',
       'Mexico', 'South Korea', 'Singapore', 'Chile', 'China', 'Malaysia',
       'Israel', 'Greece', 'Colombia', 'Hungary', 'Russia', 'Thailand',
       'Guernsey', 'Bermuda', 'Peru', 'Pakistan', 'Swaziland',
       'Indonesia', 'Nigeria', 'Kenya', 'Zimbabwe', 'Paraguay',
       'Philippines', 'Poland', 'United Arab Emirates', 'Slovenia',
       'Egypt', 'Venezuela', 'Romania', 'Slovakia', 'Cyprus',
       'Costa Rica', 'El Salvador', 'Iceland', 'Jamaica', 'Honduras',
       'Belarus', 'Vietnam', 'Ecuador', 'Czech Republic', 'Ghana', 'Fiji',
       'Panama', 'Guatemala',
       'United Kingdom of Great Britain and Northern Ireland',


In [46]:
# Normalize incoporated country
df['incorporated_country'] = df['incorporated_country'].replace('Taiwan', 'China')
df['incorporated_country'] = df['incorporated_country'].replace('Bolivia (Plurinational State of)', 'Bolivia')
df['incorporated_country'] = df['incorporated_country'].replace('Ã…land Islands', 'Finland')
df['incorporated_country'] = df['incorporated_country'].replace('China, Hong Kong Special Administrative Region', 'China')
df['incorporated_country'] = df['incorporated_country'].replace('China, Macao Special Administrative Region', 'China')
df['incorporated_country'] = df['incorporated_country'].replace('Czechia', 'Czech Republic')
df['incorporated_country'] = df['incorporated_country'].replace('Faroe Islands', 'Denmark')
df['incorporated_country'] = df['incorporated_country'].replace('Guernsey', 'United Kingdom')
df['incorporated_country'] = df['incorporated_country'].replace('Hong Kong', 'China')
df['incorporated_country'] = df['incorporated_country'].replace('Isle of Man', 'United Kingdom')
df['incorporated_country'] = df['incorporated_country'].replace('Jersey', 'United Kingdom')
df['incorporated_country'] = df['incorporated_country'].replace('Cocos (Keeling) Islands', 'Australia')
df['incorporated_country'] = df['incorporated_country'].replace('Republic of Korea', 'South Korea')
df['incorporated_country'] = df['incorporated_country'].replace('Russian Federation', 'Russia')
df['incorporated_country'] = df['incorporated_country'].replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
df['incorporated_country'] = df['incorporated_country'].replace('United States Minor Outlying Islands', 'USA')
df['incorporated_country'] = df['incorporated_country'].replace('Venezuela (Bolivarian Republic of)', 'Venezuela')
df['incorporated_country'] = df['incorporated_country'].replace('Vietnam', 'Viet Nam')

df['incorporated_country'].unique()

array(['United Kingdom', 'Spain', 'USA', 'Italy', 'South Africa', 'India',
       'France', 'Switzerland', 'Netherlands', 'Japan', 'Brazil',
       'Finland', 'Canada', 'New Zealand', 'Turkey', 'Germany', 'Ireland',
       'Australia', 'Austria', 'Luxembourg', 'China', 'Norway', 'Sweden',
       'Portugal', 'Argentina', 'Denmark', 'Belgium', 'Mexico',
       'South Korea', 'Singapore', 'Chile', 'Malaysia', 'Israel',
       'Greece', 'Colombia', 'Hungary', 'Russia', 'Thailand', 'Bermuda',
       'Peru', 'Pakistan', 'Swaziland', 'Indonesia', 'Nigeria', 'Kenya',
       'Zimbabwe', 'Paraguay', 'Philippines', 'Poland',
       'United Arab Emirates', 'Slovenia', 'Egypt', 'Venezuela',
       'Romania', 'Slovakia', 'Cyprus', 'Costa Rica', 'El Salvador',
       'Iceland', 'Jamaica', 'Honduras', 'Belarus', 'Viet Nam', 'Ecuador',
       'Czech Republic', 'Ghana', 'Fiji', 'Panama', 'Guatemala', 'Malta',
       'Kazakhstan', 'Saudi Arabia', 'Sri Lanka', 'Cayman Islands',
       'Jordan', 'Mongolia'

### Map and Create Region Column

In [49]:
country_to_region = {
    'United Kingdom': 'Europe',
    'USA': 'North America',
    'Italy': 'Europe',
    'India': 'South Asia',
    'Spain': 'Europe',
    'France': 'Europe',
    'Switzerland': 'Europe',
    'Netherlands': 'Europe',
    'Japan': 'North Asia',
    'South Africa': 'Africa',
    'Canada': 'North America',
    'New Zealand': 'Oceania',
    'Turkey': 'Middle East',
    'Germany': 'Europe',
    'Australia': 'Oceania',
    'Luxembourg': 'Europe',
    'Taiwan': 'North Asia',
    'Norway': 'Europe',
    'Sweden': 'Europe',
    'Austria': 'Europe',
    'Brazil': 'South America',
    'Portugal': 'Europe',
    'China': 'North Asia',
    'Singapore': 'South Asia',
    'South Korea': 'North Asia',
    'Russia': 'North Asia',
    'Denmark': 'Europe',
    'Ireland': 'Europe',
    'Belgium': 'Europe',
    'Finland': 'Europe',
    'Malaysia': 'South Asia',
    'Mexico': 'North America',
    'Thailand': 'South Asia',
    'Viet Nam': 'South Asia',
    'Philippines': 'South Asia',
    'Indonesia': 'South Asia',
    'Greece': 'Europe',
    'Chile': 'South America',
    'Argentina': 'South America',
    'Peru': 'South America',
    'Colombia': 'South America',
    'Ecuador': 'South America',
    'Uruguay': 'South America',
    'Paraguay': 'South America',
    'Bolivia': 'South America',
    'Venezuela': 'South America',
    'Panama': 'Central America',
    'Costa Rica': 'Central America',
    'Honduras': 'Central America',
    'El Salvador': 'Central America',
    'Nicaragua': 'Central America',
    'Guatemala': 'Central America',
    'Belize': 'Central America',
    'Hungary': 'Europe',
    'Israel': 'Middle East',
    'Cyprus': 'Europe',
    'United Arab Emirates': 'Middle East',
    'Bermuda': 'Europe',
    'Poland': 'Europe',
    'Iceland': 'Europe',
    'Czech Republic': 'Europe',
    'Cayman Islands': 'Caribbean',
    'Egypt': 'Africa',
    'Saudi Arabia': 'Middle East',
    'Qatar': 'Middle East',
    'Cambodia': 'South Asia',
    'Estonia': 'Europe',
    'Marshall Islands': 'Oceania',
    'Ukraine': 'Europe',
    'Lithuania': 'Europe',
    'Bangladesh': 'South Asia',
    'Belarus': 'Europe',
    'Bulgaria': 'Europe',
    'Cameroon': 'Africa',
    'Dominican Republic': 'Caribbean',
    'Fiji': 'Oceania',
    'Ghana': 'Africa',
    'Guyana': 'South America',
    'Jamaica': 'Caribbean',
    'Jordan': 'Middle East',
    'Kazakhstan': 'North Asia',
    'Kenya': 'Africa',
    'Kuwait': 'Middle East',
    'Libya': 'Africa',
    'Liechtenstein': 'Europe',
    'Malta': 'Europe',
    'Mauritius': 'Africa',
    'Monaco': 'Europe',
    'Mongolia': 'North Asia',
    'Morocco': 'Africa',
    'Mozambique': 'Africa',
    'Oman': 'Middle East',
    'Pakistan': 'North Asia',
    'Romania': 'Europe',
    'San Marino': 'Europe',
    'Serbia': 'Europe',
    'Slovakia': 'Europe',
    'Slovenia': 'Europe',
    'Sri Lanka': 'South Asia',
    'Swaziland': 'Africa',
    'Zimbabwe': 'Africa',
    'Nigeria': 'Africa',
}

df['region'] = df['incorporated_country'].map(country_to_region)

df['region'].unique()

array(['Europe', 'North America', 'Africa', 'South Asia', 'North Asia',
       'South America', 'Oceania', 'Middle East', 'Central America',
       'Caribbean'], dtype=object)

In [51]:
# analyze nulls to see if any countries were missed
df['region'].isnull().sum() 

##### (if this is higher than 0, use next step to figure out which country is missing)

0

In [53]:
# figure out null regions
null = df[['region', 'incorporated_country']]
# boolean indicating which values are null
null_mask = null.isnull()
# rows with at least one null value
rows_with_null = null[null_mask.any(axis=1)]
print(rows_with_null)

Empty DataFrame
Columns: [region, incorporated_country]
Index: []


# Merging World Bank Data

#### Country Code Helper Dataframe
The WB coder (*wb.economy.coder*) is part of the WBG API, which allows you to easily get the World Bank country codes with most variations of a country's spelling.

I can simply paste the list of unique incorporated country values from above into the coder to get a mapping of the countries in the dataset to World Bank API data.

In [57]:
country_mapping = wb.economy.coder(['United Kingdom', 'Spain', 'USA', 'Italy', 'South Africa', 'India',
       'France', 'Switzerland', 'Netherlands', 'Japan', 'Brazil',
       'Finland', 'Canada', 'New Zealand', 'Turkey', 'Germany', 'Ireland',
       'Australia', 'Austria', 'Luxembourg', 'China', 'Norway', 'Sweden',
       'Portugal', 'Argentina', 'Denmark', 'Belgium', 'Mexico',
       'South Korea', 'Singapore', 'Chile', 'Malaysia', 'Israel',
       'Greece', 'Colombia', 'Hungary', 'Russia', 'Thailand', 'Bermuda',
       'Peru', 'Pakistan', 'Swaziland', 'Indonesia', 'Nigeria', 'Kenya',
       'Zimbabwe', 'Paraguay', 'Philippines', 'Poland',
       'United Arab Emirates', 'Slovenia', 'Egypt', 'Venezuela',
       'Romania', 'Slovakia', 'Cyprus', 'Costa Rica', 'El Salvador',
       'Iceland', 'Jamaica', 'Honduras', 'Belarus', 'Viet Nam', 'Ecuador',
       'Czech Republic', 'Ghana', 'Fiji', 'Panama', 'Guatemala', 'Malta',
       'Kazakhstan', 'Saudi Arabia', 'Sri Lanka', 'Cayman Islands',
       'Jordan', 'Mongolia', 'Lithuania', 'San Marino', 'Bulgaria',
       'Bangladesh', 'Bolivia', 'Serbia', 'Mozambique', 'Kuwait',
       'Mauritius', 'Cambodia', 'Uruguay', 'Qatar', 'Estonia', 'Ukraine',
       'Dominican Republic', 'Morocco', 'Libya', 'Cameroon', 'Guyana',
       'Oman', 'Monaco', 'Marshall Islands', 'Liechtenstein'])

country_mapping

ORIGINAL NAME,WBG NAME,ISO_CODE
United Kingdom,United Kingdom,GBR
Spain,Spain,ESP
USA,United States,USA
Italy,Italy,ITA
South Africa,South Africa,ZAF
India,India,IND
France,France,FRA
Switzerland,Switzerland,CHE
Netherlands,Netherlands,NLD
Japan,Japan,JPN


Now that I have the mapping, I can create the helper dataframe to join the World Bank code to the incorporated country in the dataset.

In [59]:
# convert country mapping to a dataframe
country_codes = pd.DataFrame(country_mapping, index=[0])

# transpose columns to rows
country_codes = country_codes.transpose()

# reset index
country_codes = country_codes.reset_index()

#duplicate index to column to create incorporated_country
country_codes['incorporated_country'] = country_codes['index']

# rename WB code column
country_codes.rename(columns={0: 'WB_Code'}, inplace=True)

# final helper dataframe
country_codes

,index,WB_Code,incorporated_country
0,United Kingdom,GBR,United Kingdom
1,Spain,ESP,Spain
2,USA,USA,USA
3,Italy,ITA,Italy
4,South Africa,ZAF,South Africa
...,...,...,...
94,Guyana,GUY,Guyana
95,Oman,OMN,Oman
96,Monaco,MCO,Monaco
97,Marshall Islands,MHL,Marshall Islands


In [62]:
# get all unique world bank country codes to use in the API

code_list = country_codes['WB_Code'].unique()
print(code_list)

['GBR' 'ESP' 'USA' 'ITA' 'ZAF' 'IND' 'FRA' 'CHE' 'NLD' 'JPN' 'BRA' 'FIN'
 'CAN' 'NZL' 'TUR' 'DEU' 'IRL' 'AUS' 'AUT' 'LUX' 'CHN' 'NOR' 'SWE' 'PRT'
 'ARG' 'DNK' 'BEL' 'MEX' 'KOR' 'SGP' 'CHL' 'MYS' 'ISR' 'GRC' 'COL' 'HUN'
 'RUS' 'THA' 'BMU' 'PER' 'PAK' 'SWZ' 'IDN' 'NGA' 'KEN' 'ZWE' 'PRY' 'PHL'
 'POL' 'ARE' 'SVN' 'EGY' 'VEN' 'ROU' 'SVK' 'CYP' 'CRI' 'SLV' 'ISL' 'JAM'
 'HND' 'BLR' 'VNM' 'ECU' 'CZE' 'GHA' 'FJI' 'PAN' 'GTM' 'MLT' 'KAZ' 'SAU'
 'LKA' 'CYM' 'JOR' 'MNG' 'LTU' 'SMR' 'BGR' 'BGD' 'BOL' 'SRB' 'MOZ' 'KWT'
 'MUS' 'KHM' 'URY' 'QAT' 'EST' 'UKR' 'DOM' 'MAR' 'LBY' 'CMR' 'GUY' 'OMN'
 'MCO' 'MHL' 'LIE']


## Merge GDP

The first World Bank indicator I will merge to the dataset is GDP. Merging requires a set of steps that can be re-used for any indicator within the WB API. The steps include:
1. Get the specific indicator code. The *series.info* tool allows you to easily query a keyword to find indicators in the API
2. Create a dataframe using the API, defining the indicator code, country codes, and range/interval of years
3. Transpose the dataframe so that country becomes rows and years become columns
4. Merge to the helper code dataframe to get the incorporated country
5. Melt the dataframe so that every row is a combination of country and year with the indicator as the value
6. Join the final indicator dataframe to the dataset on country and year

In [66]:
# 1. Get the specific indicator code
wb.series.info(q='GDP')

id,value
EG.GDP.PUSE.KO.PP,GDP per unit of energy use (PPP $ per kg of oil equivalent)
EG.GDP.PUSE.KO.PP.KD,GDP per unit of energy use (constant 2021 PPP $ per kg of oil equivalent)
EG.USE.COMM.GD.PP.KD,"Energy use (kg of oil equivalent) per $1,000 GDP (constant 2021 PPP)"
EN.GHG.CO2.RT.GDP.KD,Carbon intensity of GDP (kg CO2e per constant 2015 US$ of GDP)
EN.GHG.CO2.RT.GDP.PP.KD,Carbon intensity of GDP (kg CO2e per 2021 PPP $ of GDP)
NY.GDP.DEFL.KD.ZG,"Inflation, GDP deflator (annual %)"
NY.GDP.DEFL.KD.ZG.AD,"Inflation, GDP deflator: linked series (annual %)"
NY.GDP.DEFL.ZS,GDP deflator (base year varies by country)
NY.GDP.DEFL.ZS.AD,GDP deflator: linked series (base year varies by country)
NY.GDP.DISC.CN,Discrepancy in expenditure estimate of GDP (current LCU)


In [67]:
# 2. Create a dataframe using the API, defining the indicator code, country codes, and range/interval of years

gdp = wb.data.DataFrame(
# Define World Bank Indicator Code (using "GDP (current US$)")
'NY.GDP.MKTP.CD', 
# Paste list of WB country codes (all incorporated countries in dataset)
['GBR','ESP','USA','ITA','ZAF','IND','FRA','CHE','NLD','JPN','BRA','FIN'
,'CAN','NZL','TUR','DEU','IRL','AUS','AUT','LUX','NOR','SWE','PRT'
,'ARG','DNK','BEL','HKG','MEX','KOR','SGP','CHL','CHN','MYS','ISR','GRC'
,'COL','HUN','RUS','THA','BMU','PER','PAK','SWZ','IDN','NGA','KEN','ZWE'
,'PRY','PHL','POL','ARE','SVN','EGY','VEN','ROU','SVK','CYP','CRI','SLV'
,'ISL','JAM','HND','BLR','ECU','CZE','GHA','FJI','PAN','GTM','MAC','MLT'
,'VNM','KAZ','SAU','LKA','CYM','JOR','MNG','LTU','SMR','BGR','BGD','BOL'
,'SRB','MOZ','KWT','IMN','MUS','KHM','URY','QAT','EST','UKR','DOM','MAR'
,'LBY','CMR','GUY','OMN','MCO','MHL','LIE','VNM']
# Define range of years and interval (every year between 2013 and 2025)
,range(2013, 2025, 1)
, index = 'time')
gdp.head()

,ARE,ARG,AUS,AUT,BEL,BGD,BGR,BLR,BMU,BOL,...,SWZ,THA,TUR,UKR,URY,USA,VEN,VNM,ZAF,ZWE
time,,,,,,,,,,,,,,,,,,,,,
YR2013,400218529748,552025140252,1577123329411,426580502769,524097026599,149998957434,55852123990,75527558966,6465756000,30659338886,...,4418333192,420333654593,957799120008,190498811460,61337621934,16880683000000,371005379787,213708811665,400886013596,19091020000
YR2014,414105366759,526319673732,1468265356428,438556021078,537987419164,172886611654,57157782910,78812805039,6413988000,32996188013,...,4250545461,407339040198,938934609297,133503871862,61496186974,17608138000000,482359318768,233451469643,381198869776,19495519600
YR2015,370275469560,594749285413,1351296372254,379546097301,461044767545,195146608978,50811995689,56454889147,6654541000,33000198248,...,3878174404,401296238228,864313810469,91030967789,57680327999,18295019000000,NaN,239258328382,346709790459,19963120600
YR2016,369255326236,557532320663,1206836962282,393687359770,474271566740,265224515675,53964253212,47723545321,6899911000,33941126200,...,3705918654,413366349748,869682881593,93355869404,57480788390,18804913000000,NaN,257096001178,323585509674,20548678100
YR2017,390516804017,643628393281,1325582658157,414926138133,500908767352,293732446625,59309748166,54725302250,7142316000,37508642165,...,4437538381,456356813537,858988492854,112090505082,65006039995,19612102000000,NaN,281353605987,381448814653,51074660513


In [68]:
# 3. Transpose the dataframe so that country becomes rows and years become columns

# convert columns to rows
gdp = gdp.transpose()
# reset index
gdp = gdp.reset_index()
#duplicate index to column
gdp['WB_Code'] = gdp['index']
# rename columns
gdp.rename(columns={1: 'WB_Code'}, inplace=True)
# drop index
gdp.drop(columns=['index'], inplace=True)

gdp.head()

time,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022,YR2023,WB_Code
0,400218529748,414105366759,370275469560,369255326236,390516804017,427049432149,417989721734,349473015337,415178792770,502731935197,514130432653,ARE
1,552025140252,526319673732,594749285413,557532320663,643628393281,524819892360,447754683615,385740508437,486564085480,632790070063,646075277525,ARG
2,1577123329411,1468265356428,1351296372254,1206836962282,1325582658157,1427809041019,1392723834563,1328414058378,1556735770437,1690858246994,1728057316696,AUS
3,426580502769,438556021078,379546097301,393687359770,414926138133,452582294973,442983642372,434397601558,480467037339,471773629830,511685203845,AUT
4,524097026599,537987419164,461044767545,474271566740,500908767352,542638913428,536726344405,529694473502,598494036474,593438820508,644782756683,BEL


In [69]:
# 4. Merge to the helper code dataframe to get the incorporated country

gdp = pd.merge(gdp, country_codes, on='WB_Code')

# rename year columns
gdp = gdp.rename(columns={
    'YR2013': '2013',
    'YR2014': '2014',
    'YR2015': '2015',
    'YR2016': '2016',
    'YR2017': '2017',
    'YR2018': '2018',
    'YR2019': '2019',
    'YR2020': '2020',
    'YR2021': '2021',
    'YR2022': '2022',
    'YR2023': '2023'
})

# drop index
gdp.drop(columns=['index'], inplace=True)

gdp

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,WB_Code,incorporated_country
0,400218529748,414105366759,370275469560,369255326236,390516804017,427049432149,417989721734,349473015337,415178792770,502731935197,514130432653,ARE,United Arab Emirates
1,552025140252,526319673732,594749285413,557532320663,643628393281,524819892360,447754683615,385740508437,486564085480,632790070063,646075277525,ARG,Argentina
2,1577123329411,1468265356428,1351296372254,1206836962282,1325582658157,1427809041019,1392723834563,1328414058378,1556735770437,1690858246994,1728057316696,AUS,Australia
3,426580502769,438556021078,379546097301,393687359770,414926138133,452582294973,442983642372,434397601558,480467037339,471773629830,511685203845,AUT,Austria
4,524097026599,537987419164,461044767545,474271566740,500908767352,542638913428,536726344405,529694473502,598494036474,593438820508,644782756683,BEL,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,16880683000000,17608138000000,18295019000000,18804913000000,19612102000000,20656516000000,21539982000000,21354105000000,23681171000000,26006893000000,27720709000000,USA,USA
95,371005379787,482359318768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VEN,Venezuela
96,213708811665,233451469643,239258328382,257096001178,281353605987,310106478395,334365270497,346615738538,366474752771,410324028883,429716969044,VNM,Viet Nam
97,400886013596,381198869776,346709790459,323585509674,381448814653,405260723893,389330032224,337974655408,420886877629,406920004594,380699271815,ZAF,South Africa


In [71]:
# 5. Melt the dataframe so that every row is a combination of country and year with the indicator as the value

melted_gdp = gdp.melt(
    id_vars=['incorporated_country','WB_Code'],          # Keep Country and WB_Code as identifier variable
    value_vars=['2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023'],  # Year columns to pivot
    var_name='Year',        # Name for the new column containing old column names
    value_name='country_gdp'         # Name for the new column containing values
)
# convert Year to int64
melted_gdp['Year'] = melted_gdp['Year'].astype('int64')
melted_gdp

,incorporated_country,WB_Code,Year,country_gdp
0,United Arab Emirates,ARE,2013,400218529748
1,Argentina,ARG,2013,552025140252
2,Australia,AUS,2013,1577123329411
3,Austria,AUT,2013,426580502769
4,Belgium,BEL,2013,524097026599
...,...,...,...,...
1084,USA,USA,2023,27720709000000
1085,Venezuela,VEN,2023,NaN
1086,Viet Nam,VNM,2023,429716969044
1087,South Africa,ZAF,2023,380699271815


In [72]:
# 6. Join the final indicator dataframe to the dataset on country and year

merged_df = df.merge(melted_gdp, on=['incorporated_country','Year'], how='inner')
merged_df

,account_id,account_name,incorporated_country,ticker,Evaluation_status,Scope_3_emissions_type,Scope_3_emissions_amount,Year,Market_Cap,Employee_Count,...,longTermInvestments_USD,totalAssets_USD,totalLiabilities_USD,totalInvestments_USD,totalDebt_USD,totalEquity_USD,employeeCount,region,WB_Code,country_gdp
0,44,3i Group,United Kingdom,III LN,"Relevant, calculated",Waste generated in operations,12,2013,3654277761,NaN,...,4460240000,5572760000,1371600000,4460240000,1078230000,4201160000,NaN,Europe,GBR,2784853502534
1,44,3i Group,United Kingdom,III LN,"Relevant, calculated",Business travel,57,2013,3654277761,NaN,...,4460240000,5572760000,1371600000,4460240000,1078230000,4201160000,NaN,Europe,GBR,2784853502534
2,78,Aberdeen Asset Management,United Kingdom,ADN LN,"Relevant, calculated",Business travel,3055,2013,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,GBR,2784853502534
3,78,Aberdeen Asset Management,United Kingdom,ADN LN,"Relevant, calculated",Downstream leased assets,1497,2013,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,GBR,2784853502534
4,180,Admiral Group,United Kingdom,ADM LN,"Relevant, calculated",Waste generated in operations,284,2013,3606629761,NaN,...,2042795000,4411726000,54102000,2042795000,127000,665607000,NaN,Europe,GBR,2784853502534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181321,927512,LGT Private Banking,Liechtenstein,NaN,"Relevant, calculated",Purchased goods and services,131,2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,LIE,NaN
181322,927512,LGT Private Banking,Liechtenstein,NaN,"Relevant, calculated",Fuel-and-energy-related activities (not includ...,947,2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,LIE,NaN
181323,927512,LGT Private Banking,Liechtenstein,NaN,"Relevant, calculated",Waste generated in operations,8,2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,LIE,NaN
181324,927512,LGT Private Banking,Liechtenstein,NaN,"Relevant, calculated",Business travel,3469,2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,LIE,NaN


### Merge Greenhouse Gas Emissions

In [74]:
# find total emissions codes
wb.series.info(q='Total greenhouse gas emissions')

id,value
EN.GHG.ALL.LU.MT.CE.AR5,Total greenhouse gas emissions including LULUCF (Mt CO2e)
EN.GHG.ALL.MT.CE.AR5,Total greenhouse gas emissions excluding LULUCF (Mt CO2e)
EN.GHG.ALL.PC.CE.AR5,Total greenhouse gas emissions excluding LULUCF per capita (t CO2e/capita)
EN.GHG.TOT.ZG.AR5,Total greenhouse gas emissions excluding LULUCF (% change from 1990)
,4 elements


In [75]:
# get total greenhouse gas emissions for all incorporated countries

country_total_ghg = wb.data.DataFrame(
# World Bank Indicator Code (using "Total greenhouse gas emissions including LULUCF (Mt CO2e)")
'EN.GHG.ALL.LU.MT.CE.AR5', 
# country codes
['GBR','ESP','USA','ITA','ZAF','IND','FRA','CHE','NLD','JPN','BRA','FIN'
,'CAN','NZL','TUR','DEU','IRL','AUS','AUT','LUX','NOR','SWE','PRT'
,'ARG','DNK','BEL','HKG','MEX','KOR','SGP','CHL','CHN','MYS','ISR','GRC'
,'COL','HUN','RUS','THA','BMU','PER','PAK','SWZ','IDN','NGA','KEN','ZWE'
,'PRY','PHL','POL','ARE','SVN','EGY','VEN','ROU','SVK','CYP','CRI','SLV'
,'ISL','JAM','HND','BLR','ECU','CZE','GHA','FJI','PAN','GTM','MAC','MLT'
,'VNM','KAZ','SAU','LKA','CYM','JOR','MNG','LTU','SMR','BGR','BGD','BOL'
,'SRB','MOZ','KWT','IMN','MUS','KHM','URY','QAT','EST','UKR','DOM','MAR'
,'LBY','CMR','GUY','OMN','MCO','MHL','LIE']
# range of years and interval
,range(2013, 2025, 1)
, index = 'time')

# repeat steps to create dataframe
country_total_ghg = country_total_ghg.transpose()
country_total_ghg = country_total_ghg.reset_index()
country_total_ghg['WB_Code'] = country_total_ghg['index']
country_total_ghg.rename(columns={1: 'WB_Code'}, inplace=True)
country_total_ghg.drop(columns=['index'], inplace=True)
country_total_ghg.head()

# repeat steps to merge country code helper
country_total_ghg = pd.merge(country_total_ghg, country_codes, on='WB_Code')

# rename year columns
country_total_ghg = country_total_ghg.rename(columns={
    'YR2013': '2013',
    'YR2014': '2014',
    'YR2015': '2015',
    'YR2016': '2016',
    'YR2017': '2017',
    'YR2018': '2018',
    'YR2019': '2019',
    'YR2020': '2020',
    'YR2021': '2021',
    'YR2022': '2022',
    'YR2023': '2023'
})

# drop index
country_total_ghg.drop(columns=['index'], inplace=True)

# Melt the dataframe
melted_ghg = country_total_ghg.melt(
    id_vars=['incorporated_country'],          #Keep 'incorporated_country' as identifier variable
    value_vars=['2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023'],  # Year columns to pivot
    var_name='Year',        # Name for the new column containing old column names
    value_name='country_total_ghg'         # Name for the new column containing values
)
# convert Year to int64
melted_ghg['Year'] = melted_ghg['Year'].astype('int64')

melted_ghg.head()

,incorporated_country,Year,country_total_ghg
0,United Arab Emirates,2013,229
1,Argentina,2013,443
2,Australia,2013,604
3,Austria,2013,82
4,Belgium,2013,128


In [76]:
# merge GHG to dataset
merged_df = merged_df.merge(melted_ghg, on=['incorporated_country','Year'], how='inner')
merged_df.head()

,account_id,account_name,incorporated_country,ticker,Evaluation_status,Scope_3_emissions_type,Scope_3_emissions_amount,Year,Market_Cap,Employee_Count,...,totalAssets_USD,totalLiabilities_USD,totalInvestments_USD,totalDebt_USD,totalEquity_USD,employeeCount,region,WB_Code,country_gdp,country_total_ghg
0,44,3i Group,United Kingdom,III LN,"Relevant, calculated",Waste generated in operations,12,2013,3654277761,NaN,...,5572760000,1371600000,4460240000,1078230000,4201160000,NaN,Europe,GBR,2784853502534,553
1,44,3i Group,United Kingdom,III LN,"Relevant, calculated",Business travel,57,2013,3654277761,NaN,...,5572760000,1371600000,4460240000,1078230000,4201160000,NaN,Europe,GBR,2784853502534,553
2,78,Aberdeen Asset Management,United Kingdom,ADN LN,"Relevant, calculated",Business travel,3055,2013,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Europe,GBR,2784853502534,553
3,78,Aberdeen Asset Management,United Kingdom,ADN LN,"Relevant, calculated",Downstream leased assets,1497,2013,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Europe,GBR,2784853502534,553
4,180,Admiral Group,United Kingdom,ADM LN,"Relevant, calculated",Waste generated in operations,284,2013,3606629761,NaN,...,4411726000,54102000,2042795000,127000,665607000,NaN,Europe,GBR,2784853502534,553


### Merge Population

In [78]:
# find total emissions codes
wb.series.info(q='population, total')

id,value
SP.POP.TOTL,"Population, total"
,1 elements


In [79]:
# get country populations

pop = wb.data.DataFrame(
# World Bank Indicator Code (using "Population, total" - SP.POP.TOTL)
'SP.POP.TOTL', 
# country codes
['GBR','ESP','USA','ITA','ZAF','IND','FRA','CHE','NLD','JPN','BRA','FIN'
,'CAN','NZL','TUR','DEU','IRL','AUS','AUT','LUX','NOR','SWE','PRT'
,'ARG','DNK','BEL','HKG','MEX','KOR','SGP','CHL','CHN','MYS','ISR','GRC'
,'COL','HUN','RUS','THA','BMU','PER','PAK','SWZ','IDN','NGA','KEN','ZWE'
,'PRY','PHL','POL','ARE','SVN','EGY','VEN','ROU','SVK','CYP','CRI','SLV'
,'ISL','JAM','HND','BLR','ECU','CZE','GHA','FJI','PAN','GTM','MAC','MLT'
,'VNM','KAZ','SAU','LKA','CYM','JOR','MNG','LTU','SMR','BGR','BGD','BOL'
,'SRB','MOZ','KWT','IMN','MUS','KHM','URY','QAT','EST','UKR','DOM','MAR'
,'LBY','CMR','GUY','OMN','MCO','MHL','LIE']
# range of years and interval
,range(2013, 2025, 1)
, index = 'time')

# repeat steps to create dataframe
pop = pop.transpose()
pop = pop.reset_index()
pop['WB_Code'] = pop['index']
pop.rename(columns={1: 'WB_Code'}, inplace=True)
pop.drop(columns=['index'], inplace=True)
pop.head()

# repeat steps to merge code helper
country_pop = pd.merge(pop, country_codes, on='WB_Code')

# rename year columns
country_pop = country_pop.rename(columns={
    'YR2013': '2013',
    'YR2014': '2014',
    'YR2015': '2015',
    'YR2016': '2016',
    'YR2017': '2017',
    'YR2018': '2018',
    'YR2019': '2019',
    'YR2020': '2020',
    'YR2021': '2021',
    'YR2022': '2022',
    'YR2023': '2023'
})

# drop index
country_pop.drop(columns=['index'], inplace=True)

# Melt the dataframe
melted_pop = country_pop.melt(
    id_vars=['incorporated_country'],          #Keep 'incorporated_country' as identifier variable
    value_vars=['2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023'],  # Year columns to pivot
    var_name='Year',        # Name for the new column containing old column names
    value_name='country_population'         # Name for the new column containing values
)
# convert Year to int64
melted_pop['Year'] = melted_pop['Year'].astype('int64')
melted_pop.head()

,incorporated_country,Year,country_population
0,United Arab Emirates,2013,7693031
1,Argentina,2013,42582455
2,Australia,2013,23128129
3,Austria,2013,8479823
4,Belgium,2013,11159407


In [80]:
# merge population to dataset
merged_final = merged_df.merge(melted_pop, on=['incorporated_country','Year'], how='inner')
merged_final

,account_id,account_name,incorporated_country,ticker,Evaluation_status,Scope_3_emissions_type,Scope_3_emissions_amount,Year,Market_Cap,Employee_Count,...,totalLiabilities_USD,totalInvestments_USD,totalDebt_USD,totalEquity_USD,employeeCount,region,WB_Code,country_gdp,country_total_ghg,country_population
0,44,3i Group,United Kingdom,III LN,"Relevant, calculated",Waste generated in operations,12,2013,3654277761,NaN,...,1371600000,4460240000,1078230000,4201160000,NaN,Europe,GBR,2784853502534,553,64128273
1,44,3i Group,United Kingdom,III LN,"Relevant, calculated",Business travel,57,2013,3654277761,NaN,...,1371600000,4460240000,1078230000,4201160000,NaN,Europe,GBR,2784853502534,553,64128273
2,78,Aberdeen Asset Management,United Kingdom,ADN LN,"Relevant, calculated",Business travel,3055,2013,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe,GBR,2784853502534,553,64128273
3,78,Aberdeen Asset Management,United Kingdom,ADN LN,"Relevant, calculated",Downstream leased assets,1497,2013,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe,GBR,2784853502534,553,64128273
4,180,Admiral Group,United Kingdom,ADM LN,"Relevant, calculated",Waste generated in operations,284,2013,3606629761,NaN,...,54102000,2042795000,127000,665607000,NaN,Europe,GBR,2784853502534,553,64128273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181321,927512,LGT Private Banking,Liechtenstein,NaN,"Relevant, calculated",Purchased goods and services,131,2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe,LIE,NaN,NaN,39850
181322,927512,LGT Private Banking,Liechtenstein,NaN,"Relevant, calculated",Fuel-and-energy-related activities (not includ...,947,2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe,LIE,NaN,NaN,39850
181323,927512,LGT Private Banking,Liechtenstein,NaN,"Relevant, calculated",Waste generated in operations,8,2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe,LIE,NaN,NaN,39850
181324,927512,LGT Private Banking,Liechtenstein,NaN,"Relevant, calculated",Business travel,3469,2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe,LIE,NaN,NaN,39850


### EXPORT FINAL DATA

In [82]:
# to csv
merged_final.to_csv('merged_data.csv', index=False)

In [83]:
# compressed csv
df.to_csv('merged_data.csv.gz', compression='gzip', index=False)